In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from numpy import array
import ast
import h5py

In [2]:
# Import the CSV file containing HN titles, mean vectors and scores
df = pd.read_hdf('hn_title_mean_vectors_scores.h5', key='df_clean')

df.head()

,id,score,mean_vector
0,3150000,19,"[0.00077195687, 0.00042079733, 0.00012249911, ..."
1,3500001,1,"[-0.0017264247, -0.00029120408, 0.0007486603, ..."
2,3150001,2,"[0.0006859072, 1.0250059e-05, 0.0010865574, -0..."
3,6050000,1,"[-0.00013478259, -0.0004138446, -0.0004691301,..."
4,150000,1,"[0.00026220054, -0.0013449227, 0.0005244772, -..."


In [77]:
df.shape

(99182, 4)

In [10]:
# Create dictionary in matrix and score format
data = {
    "matrix": df['mean_vector'].tolist(),
    "score": df['score'].values
}
dataframe = pd.DataFrame(data)


In [6]:
# #Generating random data to test training
# data = {
#     "matrix": [np.random.rand(256, 1) for _ in range(10000)],
#     "score": np.random.rand(10000),
#     }
# dataframe = pd.DataFrame(data)

In [7]:
dataframe.head()

,matrix,score
0,"[[0.07713149956433529], [0.3551647154941271], ...",0.204583
1,"[[0.1690183103348072], [0.9046856919847626], [...",0.087802
2,"[[0.4775935110488211], [0.7795798691007657], [...",0.429929
3,"[[0.02972295115012913], [0.8650728475654031], ...",0.652238
4,"[[0.3217942936974091], [0.641035480397385], [0...",0.550710


In [11]:


class RandomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Extract the matrix and score
        matrix = self.dataframe.iloc[idx]["matrix"]
        score = self.dataframe.iloc[idx]["score"]
        
        # Convert to PyTorch tensors
        matrix_tensor = torch.tensor(matrix, dtype=torch.float32).squeeze() 
        score_tensor = torch.tensor(score, dtype=torch.float32)
        
        return matrix_tensor, score_tensor

# Create Dataset
dataset = RandomDataset(dataframe)
#DODGY

# Defining DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(64, 1),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
criterion = nn.MSELoss()



In [12]:


epoch_losses = []

model = NeuralNetwork()
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)
epochs=20
for epoch in range(epochs):
    epoch_loss = 0.0
    for batch in dataloader:
        # Get the input matrix and target score
        inputs, targets = batch
        
        # Forward pass
        predictions = model(inputs)
        loss = criterion(predictions.squeeze(), targets)  # Squeeze to match dimensions
        
        # Backward pass and optimisation
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    epoch_losses.append(epoch_loss)




print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

epoch_losses = [round(loss, 4) for loss in epoch_losses]
epoch_losses


Epoch [20/20], Loss: 38.8665


[1369.8733,
 1365.763,
 1366.2659,
 1365.1431,
 1364.4442,
 1362.7783,
 1361.1803,
 1360.2452,
 1359.6838,
 1359.277,
 1359.2263,
 1358.9636,
 1358.5545,
 1358.312,
 1357.214,
 1357.8838,
 1357.255,
 1356.0181,
 1356.1941,
 1355.681]

In [84]:
epoch_losses[-20:]


[0.0891,
 0.0861,
 0.0858,
 0.085,
 0.0846,
 0.0844,
 0.0838,
 0.084,
 0.0834,
 0.0835,
 0.0833,
 0.0837,
 0.0832,
 0.0832,
 0.0833,
 0.083,
 0.083,
 0.083,
 0.0829,
 0.0826]